In [1]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [2]:
import sys
sys.path.append('..')

from langchain_community.document_loaders import PyPDFLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "，", "。", "【", ","])


In [3]:
from tools.graph_builder import TwlfGraphBuilder
graph.query('MATCH (n) DETACH DELETE n;')

graph_builder = TwlfGraphBuilder(graph)

dir = '../data'
for filename in os.listdir(dir):
    docs = []
    if filename.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(dir, filename))
        doc_pages = loader.load()
        graph_builder.graph_build(doc_pages, text_splitter)

In [4]:
from langchain_community.vectorstores import Neo4jVector
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="llama3.1",
)
graph_store = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='Chunk', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])

In [5]:
from langchain.chains import GraphCypherQAChain
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model="llama3.1",
)
graph.refresh_schema()
graph_tradition_chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True)
graph_tradition_tool = graph_tradition_chain.as_tool()
graph_tradition_tool


/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


StructuredTool(name='GraphCypherQAChain', description="Takes {'title': 'ChainInput', 'type': 'object', 'properties': {'query': {'title': 'Query'}}}.", args_schema=<class 'pydantic.v1.main.ChainInput'>, func=<function convert_runnable_to_tool.<locals>.invoke_wrapper at 0x132488fe0>, coroutine=<function convert_runnable_to_tool.<locals>.ainvoke_wrapper at 0x132488e00>)

In [6]:
vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='Chunk', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)
retriever

VectorStoreRetriever(tags=['Neo4jVector', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.neo4j_vector.Neo4jVector object at 0x121f2ffe0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.8})

In [7]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [18]:
# from langchain_core.prompts import ChatPromptTemplate
# hyde_prompt = ChatPromptTemplate.from_messages([
#     ('system', '請根據問題, 寫出300字的文章'),
#     ('user', '{text}')
# ])
# hyde_chain = (
#     hyde_prompt | llm | StrOutputParser()
# )

In [19]:
# hyde_chain.invoke("未滿15歲死亡，被保險人身故保險金怎麼賠償?")

'在大多數國家，保險公司通常會根據保險合同的條款來處理未滿15歲兒童的死亡索賠。一般而言，未滿15歲的兒童不被視為成年人，因此可能沒有完整的保險金賠償。\n\n在台灣，根據《人身保险法》第十九條之一，對未滿十五歲之死亡，只能支付保單總額百分之三十。這意味著，如果您購買了一份保險金100萬元的保單，那麼對未滿15歲的兒童的死亡索賠，最多只能收到30萬元。\n\n需要注意的是，保險公司可能會根據具體情況和合同條款進行變化。因此，如果您有類似的問題，我們強烈建議您直接與保險公司聯繫，詢問有關未滿15歲兒童的死亡索賠相關事宜。\n\n另外，也要記得檢視您的保險合同，以確定是否已經涵蓋了這方面的情況。'

In [16]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    "你是一個有用的助手, 你的任務是回答問題."
    "你必須根據以下提供的檢索內容及資料庫查詢結果進行問答問題."
    "優先使用資料庫查詢, 無法回答時採用檢索內容進行回答"
    "如果檢索內容為空, 則回答 '沒有找到相關資訊'"
    "以 5 至 10 句話以內回應, 保持答案的簡潔"
    "以下為檢索內容:\n\n"
    "{context}"

    "以下為資料庫查詢結果:\n\n"
    "{tradition_result}"

    問題: {question}
    """
)


rag_chain = (
    # hyde_chain |
    {"context": retriever | format_docs, "question": RunnablePassthrough(), "tradition_result": graph_tradition_chain}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
question = "未滿15歲被保險人死亡，身故保險金該怎麼賠?"

In [10]:
retriever.invoke(question)

[Document(metadata={'page_num': 4}, page_content='\ncontent: 第 4 頁，共  4 頁 二、被保險人故意自殺或自成失能。但自契約訂立或復效之日起二年後故意自殺致死者，本公司\n仍負給付身故保險金或 喪葬費用保險金之責任。  \n三、被保險人因犯罪處死或拒捕或越獄致死或失能。  \n因前項各款情形而免給付保險金者，本契約累積達有保單價值準備金時，依照約定給付保單價值\n準備金予應得之人。  \n \n \n（詳細內容請參閱保單條款）'),
 Document(metadata={'page_num': 3}, page_content='\ncontent: 第 3 頁，共  3 頁 本公司依 保單條款 第十六條約定計算之完全失能保險金扣除被保險人之指定保險金後，倘有餘額時，\n應將該餘額一次給付予被保險人。  \n【除外責任 】 \n【除外責任】  \n第二十五條 \n有下列情形之一者， 本公司不負給付保險金的責任。  \n一、要保人故意致被保險人於死。  \n二、被保險人故意自殺或自成完全失能。但自契約訂立或復效之日起二年後故意自殺致死者，本公司\n仍負給付身故保險金或喪葬費用保險金之責任。  \n三、被保險人因犯罪處死或拒捕或越獄致死或完全失能。  \n前項第一款及 保單條款 第二十六條情形致被保險人完全失能時，本公司按 保單條款 第十六條的約定給\n付完全失能保險金。  \n因第一項各款情形而免給付保險金者，本契約累積達有保單價值準備金時，依照約定給付保單價值準\n備金予應得之人。  \n \n（詳細內容請參閱保單條款）'),
 Document(metadata={'page_num': 3}, page_content='\ncontent: 第 3 頁，共  3 頁 【除外責任 】 \n【除外責任】  \n有下列情形之一者，本公司不負給付身故保險金 或喪葬費用保險金 及豁免保險費的責任 ： \n一、要保人故意致被保險人於死。  \n二、被保險人故意自殺或自成失能。但自契約訂立或復效之日起二年後故意自殺致死者，本公司仍負給付身故\n保險金或喪葬費用保險金之責 任。  \n三、被保險人因犯罪處死或拒捕或越獄致死或失能。  \n因前項各款情形而免給付保險金者，本契約累積達有保單價值準備金時，依照約定給

In [11]:
graph_tradition_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Chunk)-[:NEXT]->(n) RETURN c.content, n.content
Full Context:
[{'c.content': '第 1 頁，共  4 頁 台灣人壽 美鑫美利美元利率變動型終身壽險  \n保險商品內容說明  \n【承保範圍】  \n【增值回饋分享金的給付及通知】  \n第十三條  \n本公司於本契約有效期間內之每一保單年度屆滿後，除被保險人保險年齡達 16歲前者，按第三\n項約定辦理外，將依要保人於投保時所選擇下列方式之一給付增值回饋分享金：  \n一、購買增額繳清保險金額：選擇購買增額繳清保險金額者，以增值回饋分享金為躉繳純保險費\n，計算自該保單週年日當日起生效之增額繳清保險金額，但被保險人為受監護宣告尚未撤銷者，\n應依保單條款 第十六條約定辦理。  \n二、現金給付：選擇現金給付者，依本契約約定 以現金給付增值回饋分享金予要保人，惟須於第\n六保單年度屆滿後之每一保單週年日起，始得依本款方式給付。  \n三、儲存生息：選擇儲存生息者，各年度之增值回饋分享金將按各保單週年日當月之宣告利率依\n據年複利方式，累積至要保人請求時給付，或至被保險人身故或本契約終止時，由本公司主動一\n併給付。但在本公司給付受益人保險金而終止契約的情形，要保人未請求之增值回饋分享金及其\n孳息，由該保險金受益人受領。惟須於第六保單年度屆滿後之每一保單週年日起，始得依本款方\n式給付。  \n要保人若未選擇者，則視為選擇購買增額繳清保險金額，並得於本契約有效 期間內，以書面通知\n本公司變更前項給付方式，惟第六保單年度屆滿前，增值回饋分享金限以購買增額繳清保險金額\n辦理。 \n被保險人保險年齡到達 16歲前，其增值回饋分享金於繳費期間應採抵繳保險費之方式辦理。但\n因繳費期間已屆滿而無法抵繳保險費者，本公司改按各保單週年日當月之宣告利率，以年複利方\n式儲存生息至被保險人保險年齡到達 16歲時，以累計儲存生息之金額一次計算增額繳清保險金\n額，其後保單年度適用第一項規定。  \n要保人終止本契約，或被保險人於保險年齡達 16歲前死亡者，本公司應退還歷年累計儲存生息\n之金額予要保人。  \n本公司於每

{'query': '未滿15歲被保險人死亡，身故保險金該怎麼賠?',
 'result': '依據保單條款，未滿15歲被保險人的身故保險金，不包括變更為喪葬費用保險金，由本公司一次給付予被保險人父母或法定監護人。'}

In [18]:
rag_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c1:Chunk)-[r:NEXT]->(c2:Chunk) WHERE c1.page_num < 15 AND c1.id = 'insurance_policy' RETURN c1, r, c2;
Full Context:
[]

> Finished chain.


'根據提供的檢索內容和資料庫查詢結果，我們可以看到第二十五條第2款的情況下，未滿15歲被保險人故意自殺或自成完全失能，則本公司不負給付身故保險金或喪葬費用保險金的責任。但是，這個情況並沒有提及未滿15歲被保險人的死亡。\n\n另一方面，第一十五條第2款指出，被保險人因犯罪處死或拒捕或越獄致死或完全失能者，本公司不負給付身故保險金的責任。但是，這個情況也沒有提及未滿15歲被保險人的死亡。\n\n因此，根據提供的檢索內容和資料庫查詢結果，我們無法直接找到未滿15歲被保險人死亡的情況下的身故保險金支付方式。'